### Word Ngram Sentence Data Prepare And Select

In [38]:
#!pip install openpyxl

In [39]:
import os
import unicodedata
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [40]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [41]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [42]:
folder_name = "English"  # English, Turkish, German, French, Spanish, Arabic, Portuguese, Dutch, Italian
 
if folder_name == "English":
    custom_alp = en
elif folder_name == "Turkish":
    custom_alp = tr
elif folder_name == "German":
    custom_alp = de
elif folder_name == "French":
    custom_alp = fr
elif folder_name == "Spanish":
    custom_alp = es
elif folder_name == "Arabic":
    custom_alp = ar
elif folder_name == "Portuguese":
    custom_alp = pt
elif folder_name == "Dutch":
    custom_alp = nl
elif folder_name == "Italian":
    custom_alp = it
else:
    pass
#custom_alp = tr  # en, tr, de, fr, es, ar, pt, nl, it
print(f"{folder_name}\n{custom_alp}") 

English
re.compile('[abcdefghıijklmnopqrstxuvwyz]+', re.IGNORECASE)


In [43]:
def convert_one_character_letter(text):
    '''
    This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [44]:
def clean_text(text, custom_alp):  # def clean_text(text) for word
    '''
    clean_text(text, custom_alp) text is any string word or sentence.\n
    function clean string according to specific language alphabet\n
    '''
    #text_clean = re.findall(custom_alp, text)
    text_one = convert_one_character_letter(text) 
    text_clean = re.findall(custom_alp, str(text_one))
    text_result = "".join(text_clean)
    return text_result

In [45]:
def clean_text2(text, custom_alp):  # for ngram and sentence
    '''
    clean_text2(text, custom_alp) text is any string word or sentence.\n
    function clean string according to specific language alphabet\n
    '''
    #text_clean = re.findall(custom_alp, text)
    text_one = convert_one_character_letter(text) 
    text_clean = re.findall(custom_alp, str(text_one))
    text_result = " ".join(text_clean)  # difference from before
    return text_result

In [46]:
def string_length(string):
    len_out = len(string)
    return len_out

In [47]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [48]:
# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

# sentence check
twogram_sentence_check = True  # True, False
threegram_sentence_check = True
fourgram_sentence_check = True
fivegram_sentence_check = True
#sixgram_sentence_check = True
#sevengram_sentence_check = True
#eightgram_sentence_check = True
#ninegram_sentence_check = True
#tengram_sentence_check = True

# n gram select
twogram_select_start = 0
twogram_select_end = 200

threegram_select_start = 0
threegram_select_end = 200

fourgram_select_start = 0
fourgram_select_end = 200

fivegram_select_start = 0
fivegram_select_end = 200

#sixgram_select_start = 0
#sixgram_select_end = 200

#sevengram_select_start = 0
#sevengram_select_end = 200

#eightgram_select_start = 0
#eightgram_select_end = 200

#ninegram_select_start = 0
#ninegram_select_end = 200

#tengram_select_start = 0
#tengram_select_end = 200

sentence_select_start = 0
sentence_select_end = 400

In [49]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = pd.read_csv("")
df_word.drop_duplicates(inplace=True)
df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
552909,fruitcocktail,6
552910,andthesunlightshining,6
552911,upravo,6
552912,yagawa,6


In [50]:
df_word.iloc[:,0] = df_word.iloc[:,0].apply(lambda x: clean_text(x, custom_alp))
df_word.dropna(inplace=True)
df_word.drop(df_word[df_word.iloc[:,0] == ""].index, inplace=True)
#df_word.drop(df_word[df_word.iloc[:,0] == "fikret"].index, inplace=True) # paris, fikret
df_word["word"] = df_word["word"].apply(lambda x: lower_strip_func(x))
df_word.drop_duplicates(subset="word", inplace=True)
df_word.reset_index(drop=True, inplace=True)
df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
542383,fruitcocktail,6
542384,andthesunlightshining,6
542385,upravo,6
542386,yagawa,6


In [51]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [52]:
df_word = df_word[~df_word.iloc[:,0].isin(disable_list)]
df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
542383,fruitcocktail,6
542384,andthesunlightshining,6
542385,upravo,6
542386,yagawa,6


In [53]:
df_word_select = df_word.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
997,planet,178653
998,lawyer,178595
999,c,178123
1000,truck,177608


In [54]:
df_word_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Word_Selected_{word_end}.xlsx", index=False)

##### Ngrams Select

In [55]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]

df_twogram = df_twogram[df_twogram["frequency"] > 5]
df_twogram

,twogram,frequency
0,come on,1347415
1,thank you,1222974
2,all right,946314
3,i know,459647
4,excuse me,385487
...,...,...
502174,fuck luizao,6
502175,bbq conch,6
502176,bbq goose,6
502177,fuck leon,6


In [56]:
df_twogram.iloc[:,0] = df_twogram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_twogram.dropna(inplace=True)
df_twogram.drop(df_twogram[df_twogram.iloc[:,0] == ""].index, inplace=True)
df_twogram["twogram"] = df_twogram["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram.drop_duplicates(subset="twogram", inplace=True)
df_twogram.reset_index(drop=True, inplace=True)
df_twogram

,twogram,frequency
0,come on,1347415
1,thank you,1222974
2,all right,946314
3,i know,459647
4,excuse me,385487
...,...,...
494490,fuck luizao,6
494491,bbq conch,6
494492,bbq goose,6
494493,fuck leon,6


In [57]:
df_twogram_select = df_twogram.iloc[twogram_select_start:twogram_select_end,]
df_twogram_select

,twogram,frequency
0,come on,1347415
1,thank you,1222974
2,all right,946314
3,i know,459647
4,excuse me,385487
...,...,...
195,no one,19449
196,oh hey,19287
197,well i,19043
198,very nice,18992


In [58]:
df_twogram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Twogram_Selected_{twogram_select_end}.xlsx", index=False)

In [59]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,i am sorry,613528
1,let s go,480759
2,oh my god,468435
3,what is it,251203
4,that s right,228280
...,...,...
1266383,submit away adam,4
1266384,hey pool girl,4
1266385,submit it now,4
1266386,iam expecting someone,4


In [60]:
df_threegram.iloc[:,0] = df_threegram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_threegram.dropna(inplace=True)
df_threegram.drop(df_threegram[df_threegram.iloc[:,0] == ""].index, inplace=True)
df_threegram["threegram"] = df_threegram["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram.drop_duplicates(subset="threegram", inplace=True)
df_threegram.reset_index(drop=True, inplace=True)
df_threegram

,threegram,frequency
0,i am sorry,613528
1,let s go,480759
2,oh my god,468435
3,what is it,251203
4,that s right,228280
...,...,...
1255120,submit away adam,4
1255121,hey pool girl,4
1255122,submit it now,4
1255123,iam expecting someone,4


In [61]:
df_threegram_select = df_threegram.iloc[threegram_select_start:threegram_select_end,]
df_threegram_select

,threegram,frequency
0,i am sorry,613528
1,let s go,480759
2,oh my god,468435
3,what is it,251203
4,that s right,228280
...,...,...
195,who is that,17816
196,talk to me,17799
197,it s you,17638
198,am i right,17632


In [62]:
df_threegram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Threegram_Selected_{threegram_select_end}.xlsx", index=False)

In [63]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,i do not know,721942
1,what are you doing,341730
2,what do you mean,179913
3,what s going on,170653
4,what do you want,115940
...,...,...
2086918,to stop the tanks,4
2086919,corrine actually has children,4
2086920,to stop the renegade,4
2086921,to stop the rain,4


In [64]:
df_fourgram.iloc[:,0] = df_fourgram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_fourgram.dropna(inplace=True)
df_fourgram.drop(df_fourgram[df_fourgram.iloc[:,0] == ""].index, inplace=True)
df_fourgram["fourgram"] = df_fourgram["fourgram"].apply(lambda x: lower_strip_func(x))
df_fourgram.drop_duplicates(subset="fourgram", inplace=True)
df_fourgram.reset_index(drop=True, inplace=True)
df_fourgram

,fourgram,frequency
0,i do not know,721942
1,what are you doing,341730
2,what do you mean,179913
3,what s going on,170653
4,what do you want,115940
...,...,...
2077187,to stop the tanks,4
2077188,corrine actually has children,4
2077189,to stop the renegade,4
2077190,to stop the rain,4


In [65]:
df_fourgram_select = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]
df_fourgram_select

,fourgram,frequency
0,i do not know,721942
1,what are you doing,341730
2,what do you mean,179913
3,what s going on,170653
4,what do you want,115940
...,...,...
195,pleased to meet you,8882
196,yes that s right,8864
197,i can not see,8833
198,i will help you,8803


In [66]:
df_fourgram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Fourgram_Selected_{fourgram_select_end}.xlsx", index=False)

In [67]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,what are you talking about,152870
1,what are you doing here,132348
2,i do not think so,70949
3,i will be right back,47477
4,come on let s go,40716
...,...,...
2583218,he s the international press,4
2583219,contained tricyclo decenyl aloe ether,4
2583220,stay away from the animals,4
2583221,herb s an interesting guy,4


In [68]:
df_fivegram.iloc[:,0] = df_fivegram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_fivegram.dropna(inplace=True)
df_fivegram.drop(df_fivegram[df_fivegram.iloc[:,0] == ""].index, inplace=True)
df_fivegram["fivegram"] = df_fivegram["fivegram"].apply(lambda x: lower_strip_func(x))
df_fivegram.drop_duplicates(subset="fivegram", inplace=True)
df_fivegram.reset_index(drop=True, inplace=True)
df_fivegram

,fivegram,frequency
0,what are you talking about,152870
1,what are you doing here,132348
2,i do not think so,70949
3,i will be right back,47477
4,come on let s go,40716
...,...,...
2577512,he s the international press,4
2577513,contained tricyclo decenyl aloe ether,4
2577514,stay away from the animals,4
2577515,herb s an interesting guy,4


In [69]:
df_fivegram_select = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]
df_fivegram_select

,fivegram,frequency
0,what are you talking about,152870
1,what are you doing here,132348
2,i do not think so,70949
3,i will be right back,47477
4,come on let s go,40716
...,...,...
195,get out of my house,4287
196,no you do not understand,4264
197,what are you thinking about,4260
198,i will come with you,4247


In [70]:
df_fivegram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Fivegram_Selected_{fivegram_select_end}.xlsx", index=False)

##### Sentence Select

In [71]:

df_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/Sentence/Merge/Sentence_Merge.csv")  
df_sentence = df_sentence.loc[:,["sentence","frequency"]]

df_sentence

,sentence,frequency
0,no,3360789
1,yeah,3200757
2,yes,2042693
3,what,2013276
4,okay,1800201
...,...,...
16268905,tonight s party that s included,4
16268906,if the class signs on,4
16268907,her presence gives us an advantage to continue...,4
16268908,that s exactly what you can use,4


In [72]:
df_sentence.iloc[:,0] = df_sentence.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_sentence.dropna(inplace=True)
df_sentence.drop(df_sentence[df_sentence.iloc[:,0] == ""].index, inplace=True)
df_sentence["sentence"] = df_sentence["sentence"].apply(lambda x: lower_strip_func(x))
df_sentence.drop_duplicates(subset="sentence", inplace=True)
df_sentence.reset_index(drop=True, inplace=True)
df_sentence

,sentence,frequency
0,no,3360789
1,yeah,3200757
2,yes,2042693
3,what,2013276
4,okay,1800201
...,...,...
16175000,tonight s party that s included,4
16175001,if the class signs on,4
16175002,her presence gives us an advantage to continue...,4
16175003,that s exactly what you can use,4


In [73]:
df_sentence_select = df_sentence.iloc[sentence_select_start:sentence_select_end,]
df_sentence_select

,sentence,frequency
0,no,3360789
1,yeah,3200757
2,yes,2042693
3,what,2013276
4,okay,1800201
...,...,...
395,look at you,33365
396,i am okay,33353
397,about what,33167
398,there it is,33122


In [74]:
df_sentence_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Sentence_Selected_{sentence_select_end}.xlsx", index=False)